### 파이썬 미니프로젝트 연습

- 한국영진원 가입 
- 일별 박스오피스에 날짜를 바꿔가며 순위 확인해보기
- 회원가입하기
- API 키 발급 받기
- 내 키 : fad9995eb08969bb1c6cce87227a783d

# api 크롤링
기존의 크롤링 방식은
1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나(selenium)
혹은
2. 파이썬 자체적으로 호출을 요청해서(requests)<br>
데이터를 가지고 오는 방식이었습니다.<br>
이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다.<br>
따라서 서비스 제공자측에서는 서버 부하를 줄이고, 사용자에게 맞춤형을 데이터를 제공하기 위해 api서버를 운영합니다.<br>
api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며(횟수 제한이 있는 사이트도 많습니다. 라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다.)<br>
필요없는 동영상자료나 그림자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다.<br>
api 서버 접근시 보통 urllib.request 를 이용하게 됩니다.<br>

In [91]:
# 사이트에 자료 요청
import urllib.request
# json 데이터 핸들링
import json
# DataFrame 자료형 활용
import pandas as pd
# json 데이터를 pandas DataFrame으로 변환
from pandas.io.json import json_normalize

import datetime

##### 영진위 api 신청
https://www.kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do

접속 후 회원가입
- 홈페이지는 블로그 등을 기입하세요.
- 가입 후 위 사이트에서 호출요청 양식을 확인해주시면 됩니다.

In [66]:
key = 'fad9995eb08969bb1c6cce87227a783d'
target_dt = '20191231'
url = 	'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s' % (key, target_dt)
url

'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=fad9995eb08969bb1c6cce87227a783d&targetDt=20191231'

##### json데이터를 팬더스 데이터프레임으로 변환
api 데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다. <br>
쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데 <br>
파이썬의 딕셔너리와 거의 같다고 보시면 됩니다. <br>
따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상 <br>
딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다. <br>

In [67]:
# 위에 선언해둔 url을 이용하여 api 데이터 파이썬 내부로 가져오기
result = urllib.request.urlopen(url)
result

In [68]:
# 요청 페이지의 결과 데이터를 파이썬 변수에 저장
json_raw_data = result.read()


In [69]:
# .read()는  단 한번만 데이터를 풀력해주고, 두 번째부터는
# 데이터가 모두 증발해버리기 때문에 반드시 변수에 저장해야합니다.
# result.read() 2번 실행시 데이터 증발

In [70]:
# b`~~~~~` 로 시작하는 데이터는 파이썬 내부적으로 byte 자료형으로 처리됩니다.
# 데이터 프레임으로 바로 변화할수 없습니다.
json_raw_data

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20191231~20191231","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20192206","movieNm":"\xeb\xb0\xb1\xeb\x91\x90\xec\x82\xb0","openDt":"2019-12-19","salesAmt":"2530099970","salesShare":"35.4","salesInten":"692060570","salesChange":"37.7","salesAcc":"52905789770","audiCnt":"316194","audiInten":"84540","audiChange":"36.5","audiAcc":"6290502","scrnCnt":"1281","showCnt":"6056"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20184571","movieNm":"\xec\xb2\x9c\xeb\xac\xb8: \xed\x95\x98\xeb\x8a\x98\xec\x97\x90 \xeb\xac\xbb\xeb\x8a\x94\xeb\x8b\xa4","openDt":"2019-12-26","salesAmt":"1175634620","salesShare":"16.5","salesInten":"267754060","salesChange":"29.5","salesAcc":"7808242950","audiCnt":"149804","audiInten":"36229","audiChange":"31.9","audiAcc":"965624","scrnCnt":"965","showCnt":"373

In [71]:
# 바이트 자료형임을 확인
type(json_raw_data)

bytes

In [72]:
# 바이트자료형은 한글이 깨져서 나오므로 utf-8로 변경해주어야합니다.
# .decode('utf-8')을 이용합니다.
# encode -> 자연어를 기계어로 변환
# decode -> 기계어를 자연어로 변환
json_utf8 = json_raw_data.decode('utf-8')

In [73]:
# 디코드 후 b'~~~~~' 에서 b가 빠지게 됩니다.
json_utf8

'{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20191231~20191231","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20192206","movieNm":"백두산","openDt":"2019-12-19","salesAmt":"2530099970","salesShare":"35.4","salesInten":"692060570","salesChange":"37.7","salesAcc":"52905789770","audiCnt":"316194","audiInten":"84540","audiChange":"36.5","audiAcc":"6290502","scrnCnt":"1281","showCnt":"6056"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20184571","movieNm":"천문: 하늘에 묻는다","openDt":"2019-12-26","salesAmt":"1175634620","salesShare":"16.5","salesInten":"267754060","salesChange":"29.5","salesAcc":"7808242950","audiCnt":"149804","audiInten":"36229","audiChange":"31.9","audiAcc":"965624","scrnCnt":"965","showCnt":"3739"},{"rnum":"3","rank":"3","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20198414","movieNm":"시동","openDt":"2019-12-18","salesAmt":"1102216700","salesShare":"15.4","salesInten":"305970300","salesC

In [74]:
# bytes -> str로 변환확인
type(json_utf8)

str

In [75]:
# str을 dict로 변환하기 위해 json.loads(자료)를 활용
# 딕셔너리 형태의 문자열을 딕셔너리로 변환
from matplotlib.font_manager import json_load


json_complete = json.loads(json_utf8)
json_complete

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20191231~20191231',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20192206',
    'movieNm': '백두산',
    'openDt': '2019-12-19',
    'salesAmt': '2530099970',
    'salesShare': '35.4',
    'salesInten': '692060570',
    'salesChange': '37.7',
    'salesAcc': '52905789770',
    'audiCnt': '316194',
    'audiInten': '84540',
    'audiChange': '36.5',
    'audiAcc': '6290502',
    'scrnCnt': '1281',
    'showCnt': '6056'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20184571',
    'movieNm': '천문: 하늘에 묻는다',
    'openDt': '2019-12-26',
    'salesAmt': '1175634620',
    'salesShare': '16.5',
    'salesInten': '267754060',
    'salesChange': '29.5',
    'salesAcc': '7808242950',
    'audiCnt': '149804',
    'audiInten': '36229',
    'audiChange': '31.9',
    'audiAcc': '965624',
    'scrnCnt': '965',

In [76]:
# 딕셔너리로 자료형 변환 완료
type(json_complete)

dict

In [77]:
# 1차적으로 먼저 팬더스 데이터프레임화
# dailyBoxOfficeList에 실제 원하는 데이터가 있음
pd.DataFrame(json_complete)

,boxOfficeResult
boxofficeType,일별 박스오피스
dailyBoxOfficeList,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."
showRange,20191231~20191231


In [78]:
# 하단에 보면 json_complete 계층은 먼저 boxOfficeResult를 가져와야
# 하위데이터 접근가능합니다.
json_complete['boxOfficeResult']

{'boxofficeType': '일별 박스오피스',
 'showRange': '20191231~20191231',
 'dailyBoxOfficeList': [{'rnum': '1',
   'rank': '1',
   'rankInten': '0',
   'rankOldAndNew': 'OLD',
   'movieCd': '20192206',
   'movieNm': '백두산',
   'openDt': '2019-12-19',
   'salesAmt': '2530099970',
   'salesShare': '35.4',
   'salesInten': '692060570',
   'salesChange': '37.7',
   'salesAcc': '52905789770',
   'audiCnt': '316194',
   'audiInten': '84540',
   'audiChange': '36.5',
   'audiAcc': '6290502',
   'scrnCnt': '1281',
   'showCnt': '6056'},
  {'rnum': '2',
   'rank': '2',
   'rankInten': '0',
   'rankOldAndNew': 'OLD',
   'movieCd': '20184571',
   'movieNm': '천문: 하늘에 묻는다',
   'openDt': '2019-12-26',
   'salesAmt': '1175634620',
   'salesShare': '16.5',
   'salesInten': '267754060',
   'salesChange': '29.5',
   'salesAcc': '7808242950',
   'audiCnt': '149804',
   'audiInten': '36229',
   'audiChange': '31.9',
   'audiAcc': '965624',
   'scrnCnt': '965',
   'showCnt': '3739'},
  {'rnum': '3',
   'rank': '3',


In [79]:
# 먼저 boxOfficeResult에 대해 인덱싱 후
json_complete['boxOfficeResult']['dailyBoxOfficeList']

[{'rnum': '1',
  'rank': '1',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20192206',
  'movieNm': '백두산',
  'openDt': '2019-12-19',
  'salesAmt': '2530099970',
  'salesShare': '35.4',
  'salesInten': '692060570',
  'salesChange': '37.7',
  'salesAcc': '52905789770',
  'audiCnt': '316194',
  'audiInten': '84540',
  'audiChange': '36.5',
  'audiAcc': '6290502',
  'scrnCnt': '1281',
  'showCnt': '6056'},
 {'rnum': '2',
  'rank': '2',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20184571',
  'movieNm': '천문: 하늘에 묻는다',
  'openDt': '2019-12-26',
  'salesAmt': '1175634620',
  'salesShare': '16.5',
  'salesInten': '267754060',
  'salesChange': '29.5',
  'salesAcc': '7808242950',
  'audiCnt': '149804',
  'audiInten': '36229',
  'audiChange': '31.9',
  'audiAcc': '965624',
  'scrnCnt': '965',
  'showCnt': '3739'},
 {'rnum': '3',
  'rank': '3',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20198414',
  'movieNm': '시동',
  'openDt': '2019-12-18',
  'salesA

In [80]:
# 하루치 자료 데이터프레임화
daily_data = pd.DataFrame(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
daily_data

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20192206,백두산,2019-12-19,2530099970,35.4,692060570,37.7,52905789770,316194,84540,36.5,6290502,1281,6056
1,2,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,1175634620,16.5,267754060,29.5,7808242950,149804,36229,31.9,965624,965,3739
2,3,3,0,OLD,20198414,시동,2019-12-18,1102216700,15.4,305970300,38.4,21236477800,136268,37823,38.4,2525986,843,3169
3,4,4,0,NEW,20196272,미드웨이,2019-12-31,1071629520,15.0,1071629520,100,1085659520,131880,131880,100,133373,777,2587
4,5,5,-1,OLD,20197803,겨울왕국 2,2019-11-21,439575000,6.2,58382020,15.3,111596248720,55462,7058,14.6,13369064,621,1349
5,6,6,-1,OLD,20198374,신비아파트 극장판 하늘도깨비 대 요르문간드,2019-12-19,193986540,2.7,10118640,5.5,5299942440,26473,1453,5.8,672122,471,713
6,7,7,1,OLD,20192401,포드 V 페라리,2019-12-04,155205540,2.2,48285780,45.2,10612013680,18550,5728,44.7,1208407,204,373
7,8,8,3,OLD,20198430,나이브스 아웃,2019-12-04,117394600,1.6,29401780,33.4,5648860210,14139,3538,33.4,667411,158,302
8,9,9,0,OLD,20199981,눈의 여왕4,2019-12-24,78897060,1.1,2541060,3.3,1044574960,13399,2003,17.6,143853,304,347
9,10,10,-4,OLD,20192721,캣츠,2019-12-24,101775500,1.4,-50973480,-33.4,6093167690,13119,-5917,-31.1,728328,475,881


In [81]:
# 얻어온 데이터는 빈 pandas DataFrame를 만들고, 거기에 날짜 컬럼을 추가하면서 
# concat으로 누적시키면 됩니다.
final_data = pd.DataFrame(columns = ['rnum', 'rank', 'rankInten', 'rankOldAndNew',	'movieCd',
                                     'movieNm', 'openDt', 'salesAmt', 'salesShare', 'salesInten',
                                     'salesChange', 'salesAcc', 'audiCnt',	'audiInten', 'audiChange',	
                                     'audiAcc', 'scrnCnt', 'showCnt'])
final_data

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt


In [82]:
# 기존 최종데이너 + 오늘날짜를 새로운 기존 최종 데이터에 적용
# 이거 한번 더 실행하면 1~10위가 더 추가됩니다.
final_data = pd.concat([final_data, daily_data])
final_data

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20192206,백두산,2019-12-19,2530099970,35.4,692060570,37.7,52905789770,316194,84540,36.5,6290502,1281,6056
1,2,2,0,OLD,20184571,천문: 하늘에 묻는다,2019-12-26,1175634620,16.5,267754060,29.5,7808242950,149804,36229,31.9,965624,965,3739
2,3,3,0,OLD,20198414,시동,2019-12-18,1102216700,15.4,305970300,38.4,21236477800,136268,37823,38.4,2525986,843,3169
3,4,4,0,NEW,20196272,미드웨이,2019-12-31,1071629520,15.0,1071629520,100,1085659520,131880,131880,100,133373,777,2587
4,5,5,-1,OLD,20197803,겨울왕국 2,2019-11-21,439575000,6.2,58382020,15.3,111596248720,55462,7058,14.6,13369064,621,1349
5,6,6,-1,OLD,20198374,신비아파트 극장판 하늘도깨비 대 요르문간드,2019-12-19,193986540,2.7,10118640,5.5,5299942440,26473,1453,5.8,672122,471,713
6,7,7,1,OLD,20192401,포드 V 페라리,2019-12-04,155205540,2.2,48285780,45.2,10612013680,18550,5728,44.7,1208407,204,373
7,8,8,3,OLD,20198430,나이브스 아웃,2019-12-04,117394600,1.6,29401780,33.4,5648860210,14139,3538,33.4,667411,158,302
8,9,9,0,OLD,20199981,눈의 여왕4,2019-12-24,78897060,1.1,2541060,3.3,1044574960,13399,2003,17.6,143853,304,347
9,10,10,-4,OLD,20192721,캣츠,2019-12-24,101775500,1.4,-50973480,-33.4,6093167690,13119,-5917,-31.1,728328,475,881


##### 날짜 처리

- 영진위 데이터는 yyyymmdd 형식으로 처리되어있음 (2022년 10 26 -> 20221026)
- 문제는 그냥 문자열로 넘기면 20211231 + 1 이 20220101이 아닌 20211232가 되버립니다.
- 날짜타입을 따로 처리해야, 월, 년 증가, 윤년까지 처리할수있도록
- 날짜를 편하게 반복문으로 입력할 수 있습니다.

In [83]:
# 날짜 처리를 위한 datetime 임포트

from datetime import date, timedelta

In [86]:
# datetime.date(년, 월, 일)형으로 넣으면 날짜 지정이 가능합니다.
today = date(2022, 10, 26)
today

datetime.date(2022, 10, 26)

In [87]:
# datetime 날짜에 특정 년,월,일 만큼 증감하고싶다면
# timedelta(weeks = 주, days = 일)
# 년, 월을 하고 싶다면
# year = timedelta(days=365)

today + timedelta(days = 1)

datetime.date(2022, 10, 27)

In [90]:
# https://docs.python.org/ko/3/library/datetime.html#strftime-and-strptime-behavior
# 공식도큐먼트

In [89]:
# 날짜를 문자열로 변환
# %y는 2000번대 를 뺀 년도, %Y는 2000번대를 포함한 년도
# %m은 월, %d는 일을 나타냅니다.
today.strftime('%y%m%d')

'221026'

In [ ]:
# 데이터를 받아와서 시각화 api, 크롤링 소스
# 인체사이즈 